In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/zomato-bangalore-restaurants/zomato.csv')

In [ ]:
df.head()

In [ ]:
percent = (df.isnull().sum()*100/len(df)).sort_values(ascending=False)
percent

In [ ]:
df.rate.unique()

In [ ]:
df.rate = df.rate.replace('NEW',np.nan)
df.dropna(how ='any', inplace = True)

In [ ]:
del df['url']
del df['address']
del df['phone']
del df['location']
df.rename(columns={'approx_cost(for two people)': 'avg_cost', 'listed_in(city)': 'locality','listed_in(type)': 'restaurant_type'}, inplace=True)
df.head()

In [ ]:
df.rate.dtype

In [ ]:
D = df
D.rate = D.rate.astype(str)
D.rate = D.rate.apply(lambda x:x.replace('/5',''))
D.rate = D.rate.apply(lambda x:float(x))
D.rate.unique()

In [ ]:
plt.figure(figsize=(12,6))
g = sns.countplot(df['locality'])
g.set_xticklabels(g.get_xticklabels(), rotation=90, ha="right")
g

In [ ]:
plt.figure(figsize=(12,6))
g = sns.countplot(df['rest_type'])
g.set_xticklabels(g.get_xticklabels(), rotation=90, ha="right")
g

In [ ]:
plt.figure(figsize=(12,6))
D['online_order'].value_counts().plot.bar()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 9)
x = pd.crosstab(D['rate'], D['online_order'])
x.div(x.sum(1).astype(float), axis=0).plot(kind='bar',stacked=True, color=['red','yellow'])
plt.title('online order vs rate')
plt.legend(loc='upper right')
plt.show()

In [ ]:
D['book_table'].value_counts().plot.bar(color='cyan')

In [ ]:
plt.rcParams['figure.figsize'] = (15, 9)
x = pd.crosstab(D['rate'], D['book_table'])
x.div(x.sum(1).astype(float), axis=0).plot(kind='bar',stacked=True, color=['red','yellow'])
plt.title('table booking vs rate')
plt.legend(loc='upper right')
plt.show()

In [ ]:
D.head(1)

In [ ]:
D.avg_cost = D.avg_cost.apply(lambda x: x.replace(',',''))
D.avg_cost = D.avg_cost.astype(int)
sns.distplot(D.avg_cost)

In [ ]:
restaurantTypeCount = df['restaurant_type'].value_counts().sort_values(ascending=True)
slices=[restaurantTypeCount[0],
        restaurantTypeCount[1],
        restaurantTypeCount[2],
        restaurantTypeCount[3],
        restaurantTypeCount[4],
        restaurantTypeCount[5],
        restaurantTypeCount[6]]
labels=['Pubs and bars','Buffet','Drinks & nightlife','Cafes','Desserts','Dine-out','Delivery ']
colors = ['#3333cc','#ffff1a','#ff3333','#c2c2d6','#6699ff','#c4ff4d','#339933']
plt.pie(slices, colors=colors, labels=labels, autopct='%1.1f%%', pctdistance=.5, labeldistance=1.2, shadow=True)
fig = plt.gcf()
plt.title("Percentage of Restaurants according to their Type")

fig.set_size_inches(12,12)
plt.show()

In [ ]:
D = D.drop_duplicates(subset='name', keep='first')
newdf = D[['name', 'avg_cost', 'locality', 'rest_type', 'cuisines']].groupby(['avg_cost'], sort=True)
newdf = newdf.filter(lambda x:x.mean() <= 1500)
newdf = newdf.sort_values('avg_cost')

newdf_expensive=D[['name','avg_cost','locality','rest_type','cuisines']].groupby(['avg_cost'], sort = True)
newdf_expensive=newdf_expensive.filter(lambda x: x.mean() >= 3000)
newdf_expensive=newdf_expensive.sort_values(by=['avg_cost'])


In [ ]:
newdf_rate = D[['name', 'rate']].groupby('rate', sort=True)
newdf_rate = newdf_rate.filter(lambda x: x.mean()  >= 4.5)
newdf_rate = newdf_rate.sort_values('rate')
D.rate.value_counts()
D.rate.unique()
D.nunique()

In [ ]:
s1 = pd.merge(newdf, newdf_rate, how='inner', on=['name'])

s2 = pd.merge(newdf_expensive, newdf_rate, how='inner', on=['name'])

print("Cheap restaurants with low cost,high rating \n")
s1

In [ ]:
print("Expensive restaurants with high cost,high rating \n")
s2

In [ ]:
# X1.votes.describe()
newdf_votes=D[['name','votes']].groupby(['votes'], sort = True)
newdf_votes=newdf_votes.filter(lambda x: x.mean() >= 175)
newdf_votes=newdf_votes.sort_values(by=['votes'])

In [ ]:
s = pd.merge(s1, newdf_votes, how='inner', on=['name'])
s=s.sort_values(by=['avg_cost'])
print("Cheap restaurants,high rating,high votes")
s

In [ ]:
s = pd.merge(s2, newdf_votes, how='inner', on=['name'])
s=s.sort_values(by=['avg_cost'])
s